In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, make_scorer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold, learning_curve, GridSearchCV, RandomizedSearchCV

In [ ]:
# importing dataset
from google.colab import files
uploaded = files.upload()

Saving ML-CUP23-TR.csv to ML-CUP23-TR (1).csv


In [ ]:
# importing dataset
from google.colab import files
uploaded1 = files.upload()

Saving ML-CUP23-TS.csv to ML-CUP23-TS (1).csv


ho eliminato le prime 7 righe dai dataset contenenti informazioni come mail, ecc.

In [ ]:
import io
df_traincup = pd.read_csv(io.BytesIO(uploaded['ML-CUP23-TR (1).csv']), names=['id', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'target_x', 'target_y', 'target_z'], delimiter=',')

In [ ]:
import io
df_testcup = pd.read_csv(io.BytesIO(uploaded1['ML-CUP23-TS (1).csv']), names=['id', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'target_x', 'target_y', 'target_z'], delimiter=',')

In [ ]:
df_testcup

,id,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,target_x,target_y,target_z
0,1,-0.983589,0.989514,-0.998539,0.999440,0.970297,-0.234119,-0.133332,0.899576,-0.605156,0.656074,NaN,NaN,NaN
1,2,-0.296891,-0.831832,-0.967981,0.986235,0.998669,0.997767,-0.340468,0.817937,-0.968516,0.699273,NaN,NaN,NaN
2,3,-0.986512,0.979557,-0.998547,0.999430,0.985407,0.166472,0.035096,0.877812,-0.698500,0.667267,NaN,NaN,NaN
3,4,-0.594083,0.986703,-0.992113,0.999881,0.982517,0.187048,-0.135851,0.995478,-0.801041,-0.864406,NaN,NaN,NaN
4,5,0.940018,0.934761,-0.159506,0.999147,0.846696,0.413116,-0.628644,0.998797,-0.967884,-0.949134,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,896,-0.913925,0.987170,-0.996000,0.999788,0.983495,0.175686,-0.144997,0.984785,-0.722988,-0.485687,NaN,NaN,NaN
896,897,-0.066368,0.998065,-0.985424,0.999926,0.739824,-0.861055,0.618941,0.997555,-0.105691,-0.960090,NaN,NaN,NaN
897,898,-0.991811,0.995585,-0.999500,0.999681,0.449969,-0.911028,0.592227,0.551763,0.724835,0.586370,NaN,NaN,NaN
898,899,0.501175,-0.991071,0.067171,0.848283,0.992781,0.999583,0.052389,-0.720990,-0.971404,0.906668,NaN,NaN,NaN


In [ ]:
def split_data(data: pd.DataFrame, cols_name_split: list, rows_split_perc=1):
  '''
      It makes the split of the columns passed in \"cols_name_split\" and the split of the rows based on the \
      percentage \"rows_split_perc\".\n
      If \"cols_name_split\" = [] -> the method returns: (data, None).\n
      else -> (data_splitted_x, data_splitted_y) with rows_split_perc=1 \
          or (data_splitted_x_train, data_splitted_y_train, data_splitted_x_val, data_splitted_y_val) with rows_split_perc!=1.\n
      So this method makes split on columns or on rows (or both).\n\n
      Returns a tuple of two new DataFrames: (x,y).\n
      - x: is like \"df\" without the columns specified in the list \"cols_name_split\".\n
      - y: are the columns indentified by the list \"cols_name_split\".\n
      or a tuple of this format (x_train, y_train, x_val, y_val) with:
      - x_train: is like \"df\" without the columns specified in the list \"cols_name_split\".\n
      - y_train: are the columns indentified by the list \"cols_name_split\" used for Training.\n
      - x_val: is like \"df\" without the columns specified in the list \"cols_name_split\".\n
      - y_val: are the columns indentified by the list \"cols_name_split\" used for Validation.\n\n
      The parameters are:\n
      - df: the input DataFrame.\n
      - cols_name_split: list of names of target columns.\n
      - rows_split_perc: percentage of the data to split for Training, and so the 1-rows_split_perc percentage for Validation.
  '''
  # Case of no columns split
  if cols_name_split == []:
      # Case of rows split
      if rows_split_perc != 1:
          return np.split(data, [int(data.shape[0] * rows_split_perc)], axis=0)
      # Case of no rows and no columns split
      else:
          raise ValueError

  # Columns split
  y = data[cols_name_split].copy(deep=True)
  x = data.drop(columns=cols_name_split,axis=1).copy(deep=True)

  # Case of only columns split
  if rows_split_perc == 1:
      return x, y

  # Case of both splits (rows split and columns split)
  else:
      #x_train = x[:int(x.shape[0] * rows_split_perc), :]
      #x_val = x[int(x.shape[0] * rows_split_perc):, :]
      x_train, x_val = np.split(x, [int(x.shape[0] * rows_split_perc)], axis=0)
      #y_train = y[:int(x.shape[0] * rows_split_perc), :]
      #y_val = y[int(x.shape[0] * rows_split_perc):, :]
      y_train, y_val = np.split(y, [int(y.shape[0] * rows_split_perc)], axis=0)
      return x_train, y_train, x_val, y_val

## DATA SPLIT

In [ ]:
# Saving the ID columns
df_id_train: pd.DataFrame = df_traincup['id']
df_id_test: pd.DataFrame = df_testcup['id']

# Drop the ID columns
df_train = df_traincup.drop(columns=['id'],axis=1).copy(deep=True)
df_test = df_testcup.drop(columns=['id'],axis=1).copy(deep=True)

# Split of columns and rows (0.8/0.2) into: TR set and Internal TS set
x_train, y_train, x_internal_test, y_internal_test = split_data(
    data=df_train,
    cols_name_split=['target_x','target_y','target_z'],
    rows_split_perc=0.8
)

# Split on columns
x_test, y_test = split_data(data=df_test, cols_name_split=['target_x','target_y','target_z'])

# Print of the shapes
print(f"[IDs TR SET]: " + str(df_id_train.shape))
print(f"[IDs TS SET]: " + str(df_id_test.shape))
print(f"[TR SET - x]: " + str(x_train.shape))
print(f"[TR SET - y]: " + str(y_train.shape))
print(f"[Internal TS SET - x]: " + str(x_internal_test.shape))
print(f"[Internal TS SET - y]: " + str(y_internal_test.shape))
print(f"[TS SET - x]: " + str(x_test.shape))
print(f"[TS SET - y]: " + str(y_test.shape))

[IDs TR SET]: (1000,)
[IDs TS SET]: (900,)
[TR SET - x]: (800, 10)
[TR SET - y]: (800, 3)
[Internal TS SET - x]: (200, 10)
[Internal TS SET - y]: (200, 3)
[TS SET - x]: (900, 10)
[TS SET - y]: (900, 3)


## MODEL SELECTION

In [ ]:
# Define custom loss function
def euclidean_distance_loss(y_true, y_pred):
    return np.sqrt(np.sum(np.square(y_pred - y_true)))

# Define a custom scorer based on the custom loss function
def euclidean_distance_score(y_true, y_pred):
    return np.mean(euclidean_distance_loss(y_true, y_pred))

# Use make_scorer to create a scorer suitable for RandomizedSearchCV
scorer = make_scorer(euclidean_distance_score, greater_is_better=False)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
X_train = scaler.transform(x_train)
X_test = scaler.transform(x_internal_test)

In [ ]:
X_train

array([[-0.83995963, -1.06520252, -0.7609043 , ..., -0.05585995,
        -0.60723304,  0.86710607],
       [-0.76989621,  0.90343656, -0.77095157, ...,  0.78145028,
         1.49693153, -1.00037328],
       [-0.92759   ,  0.85739246, -0.77128093, ...,  0.22383702,
        -0.60112807,  1.27038832],
       ...,
       [-0.6885629 , -1.38782731,  1.58056561, ..., -2.09418299,
         1.12916939,  0.52182744],
       [-0.27213469, -1.37803163,  1.34472824, ..., -1.99652255,
        -0.63095444,  0.821477  ],
       [-0.75912636, -0.84233787, -0.76521543, ...,  0.56309343,
        -0.92684769,  0.8113172 ]])

In [ ]:
from sklearn.svm import SVR

In [ ]:
# Parameter grid for the Grid Search
param_grid = {'C': [0.0001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10], 'epsilon': [10, 1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf', 'sigmoid', 'poly'], 'degree':[2, 3, 4, 5, 6, 7, 8, 9, 10], 'gamma': ['scale', 'auto', 0.1, 0.5, 0.01, 0.05, 0.005, 0.001], 'max_iter': [340]} #no precomputed kernel perché ha bisogno di una matrice quadrata (n,n)

USATO GRID SEARCH ANCHE SE I NOMI SUGGERISCONO RANDOMIZED SEARCH


### classe x

In [ ]:
y_train_x = y_train['target_x']
y_test_x = y_internal_test['target_x']

In [ ]:
random_SVR_x = GridSearchCV(SVR(),param_grid,refit=True,verbose=0,cv=5)

In [ ]:
random_SVR_x.fit(X_train,y_train_x)

Output streaming troncato alle ultime 5000 righe.
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solv

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.0001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
                         'degree': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'epsilon': [10, 1, 0.1, 0.01, 0.001, 0.0001],
                         'gamma': ['scale', 'auto', 0.1, 0.5, 0.01, 0.05, 0.005,
                                   0.001],
                         'kernel': ['rbf', 'sigmoid', 'poly'],
                         'max_iter': [340]})

In [ ]:
print("Best parameters obtained by Random Search - label x:", random_SVR_x.best_params_)

Best parameters obtained by Random Search - label x: {'C': 10, 'degree': 2, 'epsilon': 0.0001, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 340}


In [ ]:
best_regressor_x = random_SVR_x.best_estimator_

In [ ]:
y_train_pred_x = best_regressor_x.predict(X_train)

In [ ]:
train_loss = euclidean_distance_loss(y_train_x, y_train_pred_x)
print("MEE - label x: ", train_loss)

MEE - label x:  36.55803016725437


In [ ]:
# Assuming x_val and y_val are your validation data
y_val_pred_x = best_regressor_x.predict(X_test)
val_loss_x = euclidean_distance_loss(y_test_x, y_val_pred_x)
print("val loss - label x: ", val_loss_x)

val loss - label x:  26.804678927734553


### classe y

In [ ]:
y_train_y = y_train['target_y']
y_test_y = y_internal_test['target_y']

#training
random_SVR_y = GridSearchCV(SVR(),param_grid,refit=True,verbose=0,cv=5)
random_SVR_y.fit(X_train,y_train['target_y'])


Output streaming troncato alle ultime 5000 righe.
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solv

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.0001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
                         'degree': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'epsilon': [10, 1, 0.1, 0.01, 0.001, 0.0001],
                         'gamma': ['scale', 'auto', 0.1, 0.5, 0.01, 0.05, 0.005,
                                   0.001],
                         'kernel': ['rbf', 'sigmoid', 'poly'],
                         'max_iter': [340]})

In [ ]:
print("Best parameters obtained by Random Search - label y:", random_SVR_y.best_params_)
best_regressor_y = random_SVR_y.best_estimator_
y_train_pred_y = best_regressor_y.predict(X_train)

#training validation
train_loss_y = euclidean_distance_loss(y_train_y, y_train_pred_y)
print("MEE - label y: ", train_loss_y)

Best parameters obtained by Random Search - label y: {'C': 10, 'degree': 2, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 340}
MEE - label y:  27.877532760107762


In [ ]:
#internal validation
y_val_pred_y = best_regressor_y.predict(X_test)
val_loss_y = euclidean_distance_loss(y_test_y, y_val_pred_y)
print("val loss - label y: ", val_loss_y)

val loss - label y:  16.729536328823166


### classe z

In [ ]:
y_train_z = y_train['target_z']
y_test_z = y_internal_test['target_z']

#training
random_SVR_z = GridSearchCV(SVR(),param_grid,refit=True,verbose=0,cv=5)
random_SVR_z.fit(X_train,y_train['target_z'])

Output streaming troncato alle ultime 5000 righe.
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solv

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.0001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
                         'degree': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'epsilon': [10, 1, 0.1, 0.01, 0.001, 0.0001],
                         'gamma': ['scale', 'auto', 0.1, 0.5, 0.01, 0.05, 0.005,
                                   0.001],
                         'kernel': ['rbf', 'sigmoid', 'poly'],
                         'max_iter': [340]})

In [ ]:
print("Best parameters obtained by Random Search - label z:", random_SVR_z.best_params_)
best_regressor_z = random_SVR_z.best_estimator_
y_train_pred_z = best_regressor_z.predict(X_train)

#training validation
train_loss_z = euclidean_distance_loss(y_train_z, y_train_pred_z)
print("MEE - label z: ", train_loss_z)

Best parameters obtained by Random Search - label z: {'C': 10, 'degree': 2, 'epsilon': 0.001, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': 340}
MEE - label z:  15.02615578625932


In [ ]:
#internal validation
y_val_pred_z = best_regressor_z.predict(X_test)
val_loss_z = euclidean_distance_loss(y_test_z, y_val_pred_z)
print("val loss - label z: ", val_loss_z)

val loss - label z:  9.225399944793628


### RECAP

CUP - SVR:

Best parameters obtained by Random Search - label x: {'C': 10, 'degree': 2, 'epsilon': 0.0001, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 340}
- MEE - label x:  36.55803016725437
- val loss - label x:  26.804678927734553


Best parameters obtained by Random Search - label y: {'C': 10, 'degree': 2, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 340}
- MEE - label y:  27.877532760107762
- val loss - label y:  16.729536328823166


Best parameters obtained by Random Search - label z: {'C': 10, 'degree': 2, 'epsilon': 0.001, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': 340}
- MEE - label z:  15.02615578625932
- val loss - label z:  9.225399944793628